### Implement a complete RAG system that starts with a YouTube URL, automatically transcribes it using Whisper, and then uses a retrieval-augmented generation pipeline to answer questions based on the video’s content.

In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

### Setup Your Environment

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()  # Loads variables from .env

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = os.getenv("PINECONE_API_ENV")
INDEX_NAME = os.getenv("INDEX_NAME", "youtube-transcripts")


### Download Audio from a YouTube URL

In [3]:
import os
from pytubefix import YouTube
import whisper

# Download audio from YouTube using pytubefix
youtube_url = "https://www.youtube.com/watch?v=HISRUrJsD08"
yt = YouTube(youtube_url)
audio_stream = yt.streams.filter(only_audio=True).first()
audio_file = audio_stream.download(filename="audio.mp4")
print("Audio downloaded to:", audio_file)

Audio downloaded to: d:\Work\Jupyter projects\Projects\RAG-Applications\Youtube Transcript RAG\audio.mp4


### Transcribe the Audio Using Whisper

In [4]:
# Define the transcription file name
transcription_filename = "transcript1.txt"

# If transcription file exists, load it; otherwise, transcribe and save it.
if os.path.exists(transcription_filename):
    with open(transcription_filename, "r", encoding="utf-8") as file:
        transcript = file.read()
    print("Loaded transcription from", transcription_filename)
else:
    # Load the Whisper model
    whisper_model = whisper.load_model("base")
    # Transcribe the downloaded audio file
    result = whisper_model.transcribe(audio_file)
    transcript = result["text"]
    # Save the transcription to a text file
    with open(transcription_filename, "w", encoding="utf-8") as file:
        file.write(transcript)
    print("Transcription generated and saved to", transcription_filename)

# Print a snippet of the transcript
print("Transcript snippet:", transcript[:200])

Loaded transcription from transcript1.txt
Transcript snippet:  AI agents is the biggest opportunity of 25. AI agents When you must be seeing it everywhere on social media, in news and in the startup space as well. Hi everyone, I'm Ishan Sharma and if you want to


### Split the Transcript into Chunks

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Adjust these parameters based on your transcript size and token limits
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(transcript)

print(f"Number of chunks: {len(chunks)}")
print("Sample chunk:", chunks[0][:150])


Number of chunks: 17
Sample chunk: AI agents is the biggest opportunity of 25. AI agents When you must be seeing it everywhere on social media, in news and in the startup space as well.


### Generate Embeddings and Build a Vector Store

#### Option A: Using FAISS (Local In-Memory Store)

In [6]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Create embeddings object with your OpenAI API key
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Build the FAISS vector store from text chunks
vectorstore = FAISS.from_texts(chunks, embeddings)


C:\Users\rohan\AppData\Local\Temp\ipykernel_24080\275497326.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


#### Option B: Using Pinecone (Production-Ready)

In [7]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain.vectorstores import Pinecone as LC_Pinecone
from langchain.embeddings import OpenAIEmbeddings

# Initialize the Pinecone client using the new interface
pc = Pinecone(api_key=PINECONE_API_KEY)
INDEX_NAME = "test-yt-index" 

# Optionally, check if the index exists and create it if not.
# (Make sure the dimension matches your embeddings—in this example, 1536 for OpenAI)
existing_indexes = pc.list_indexes().names()
if INDEX_NAME not in existing_indexes:
    pc.create_index(
        name=INDEX_NAME,
        dimension=1536,
        metric="euclidean",
        spec=ServerlessSpec(cloud="aws", region=PINECONE_API_ENV)
    )
    print(f"Index '{INDEX_NAME}' created.")
else:
    print(f"Index '{INDEX_NAME}' already exists.")

# Create an embeddings object with your OpenAI API key
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Assume 'chunks' is your list of text chunks.
# Build the Pinecone vector store from text chunks using LangChain's wrapper.
vectorstore = LC_Pinecone.from_texts(chunks, embeddings, index_name=INDEX_NAME)

print("Pinecone vector store created successfully.")


### Retrieve Relevant Context for a Query

In [10]:
query = "Ideas to make money using AI agents?"
retrieved_docs = vectorstore.similarity_search(query, k=3)

# Combine the retrieved document chunks into one context string
context = "\n".join([doc.page_content for doc in retrieved_docs])
print("Retrieved context:\n", context)

Retrieved context:
 you start solving those. Figure out where is the majority of the money going by businesses. I have made a separate video which goes in depth and talks about the 15 top business ideas for AI agents in 2025. So check that if you are curious to learn more about this. That's all from me today. I hope you learned a thing or two. Let me know if you have any questions below in the comment section and I'll see you in the next video. The description will have some learning resources so check those out if you're interested. There's a report that Google published which is on AI agents so read that if you're curious and I'll see you in the next one. Bye.
spending their money on hiring. They might be spending their money on operations, on finances, accounting, legal and a million other things. Figure out what are those tools which are these companies currently working with. And if you can build a AI agent version of those tools of those SaaS companies. This is from a recent Y Co

### Build the Prompt and Execute the RAG Chain

In [11]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

# Define a prompt template that uses context and question
template = """Answer the question based on the context below.
Context: {context}
Question: {question}
If you cannot answer, reply "I don't know".
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

# Initialize the model (as done in Step 1) if not already done
model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=OPENAI_API_KEY)

# Create the chain to link the prompt with the model
chain = LLMChain(llm=model, prompt=prompt)

# Execute the chain
response = chain.run(context=context, question=query)
print("Final response:", response)


Final response: Some ideas to make money using AI agents include creating AI agent versions of popular SaaS tools, offering AI agent services for tasks like data analysis and customer service, developing AI agents for specific industries or business needs, and providing consulting services for businesses looking to implement AI agents.
